In [1]:
import os
import sys
sys.path.append(os.path.join(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src')))
sys.path.append(os.path.join(
    os.environ["RNB_PLANNING_DIR"], 'src/scripts/demo_202107'))

CONNECT_INDY = False
CONNECT_MOBILE = False
USE_SDK = True

IP_CUR = "192.168.0.10"# get_ip_address()
MOBILE_IP = "192.168.0.102"
INDY_IP = "192.168.0.3"

print("Current PC IP: {}".format(IP_CUR))
print("Mobile ROB IP: {}".format(MOBILE_IP))

Current PC IP: 192.168.0.10
Mobile ROB IP: 192.168.0.102


In [2]:
from enum import Enum
class ExpType(Enum):
    REMOVE_OBS=0
    CONTACT_TAR=1

EXP_SCENARIO = ExpType.CONTACT_TAR

In [3]:
INDY_BASE_OFFSET = (0.172,0,0.439)
INDY_BASE_RPY = (0,0,0)
TOOL_NAME = "brush_face"
WALL_THICKNESS = 0.01
CLEARANCE = 0.001
WS_HEIGHT = 1.6
COL_COLOR = (1,1,1,0.2)
    
if EXP_SCENARIO == ExpType.REMOVE_OBS: ## Obstacle removing
    BAG_COUNT = 5
    CLOCK_COUNT = 0
    TARGET_COUNT = 5
    LOG_FORCE = False
else: ## Contact 
    BAG_COUNT = 3
    CLOCK_COUNT = 3
    TARGET_COUNT = 5
    LOG_FORCE = True

from pkg.controller.combined_robot import *
from pkg.project_config import *

if not CONNECT_INDY:
    indy_7dof_client.kiro_tool.OFFLINE_MODE = True
kiro_udp_client.KIRO_UDP_OFFLINE_DEBUG = not CONNECT_MOBILE

mobile_config = RobotConfig(0, RobotType.kmb, ((0,0,0), (0,0,0)),
                "{}/{}".format(MOBILE_IP, IP_CUR))
robot_config = RobotConfig(1, RobotType.indy7, 
                           (INDY_BASE_OFFSET, INDY_BASE_RPY),
                INDY_IP, root_on="kmb0_platform", 
                           specs={"no_sdk":True} if not USE_SDK else {})

ROBOT_TYPE = robot_config.type
MOBILE_NAME = mobile_config.get_indexed_name()
ROBOT_NAME = robot_config.get_indexed_name()
crob = CombinedRobot(robots_on_scene=[mobile_config, robot_config]
              , connection_list=[True, CONNECT_INDY])

connection command:
kmb0: True
indy1: False


### Make scene

In [4]:
from pkg.geometry.builder.scene_builder import SceneBuilder
from pkg.planning.scene import PlanningScene

s_builder = SceneBuilder(None)
gscene = s_builder.create_gscene(crob)
gtems = s_builder.add_robot_geometries(
    color=COL_COLOR, display=True, collision=True)

gscene.set_workspace_boundary(
    -1, 4, -2.5, 2.5, -CLEARANCE, WS_HEIGHT, thickness=WALL_THICKNESS)

Unable to register with master node [http://localhost:11311]: master may not be running yet. Will keep trying.
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker


### Planning scene

In [5]:
from pkg.geometry.geometry import *
from pkg.utils.code_scraps import *
from pkg.planning.constraint.constraint_subject import *
from pkg.planning.constraint.constraint_actor import *

In [6]:
pscene = PlanningScene(gscene, combined_robot=crob)
ROBOT_BASE = pscene.robot_chain_dict[ROBOT_NAME]['link_names'][0]
TIP_LINK = pscene.robot_chain_dict[ROBOT_NAME]["tip_link"]
CAM_LINK = TIP_LINK.replace("tcp", "link6")
MOBILE_BASE = pscene.robot_chain_dict[MOBILE_NAME]["tip_link"]
HOLD_LINK = MOBILE_BASE
BRUSH_NAME = "brush_face"
brush_face = add_indy_sweep_tool(gscene, ROBOT_NAME, face_name=BRUSH_NAME, tool_offset=0.12, color_col=(0.8,0.8,0.8,0.2))
brush_b = pscene.create_binder(BRUSH_NAME, BRUSH_NAME, WayFramer, point=(0,0,brush_face.dims[2]/2), rpy=(0,np.pi,0))
# from demo_utils.environment import add_kiro_indytool_down
# add_kiro_indytool_down(gscene, zoff=0, tool_link="indy1_tcp", face_name="brush_face", ext_off=0.032, tool_dim=(0.08, 0.32))
gscene.show_pose(crob.home_pose)

MOBILE_PUSH_NAME = "mpush"
MOBILE_PUSH_HEIGHT = 0.4
MOBILE_PUSH_KEY = 1
front_dist = 0.45+CLEARANCE
mpush_g = gscene.create_safe(GEOTYPE.SPHERE, MOBILE_PUSH_NAME, link_name=HOLD_LINK,dims=(0.01,0.01,0.01),
                           center=(front_dist, 0, MOBILE_PUSH_HEIGHT), rpy=(0,np.pi/2,0),
                           fixed=True, collision=False, color=(1,0,0,0.5))
mpush_b = pscene.create_binder(MOBILE_PUSH_NAME, MOBILE_PUSH_NAME, FramedTool, point=(0,0,0), rpy=(0,0,0), key=MOBILE_PUSH_KEY)

floor_b = pscene.create_binder("floor_ws", "floor_ws", PlaceFrame, point=(0,0,WALL_THICKNESS/2), key=MOBILE_PUSH_KEY)

TOOL_PUSH_NAME = "tpush"
TOOL_PUSH_HEIGHT = 0.03
TOOL_PUSH_KEY = 10
brush_col = gscene.NAME_DICT[BRUSH_NAME+"_col"]
TOOL_DIM = brush_col.dims
tpush_g = gscene.create_safe(GEOTYPE.SPHERE, TOOL_PUSH_NAME, link_name=TIP_LINK,dims=(0.01,0.01,0.01),
                           center=(-TOOL_DIM[0]/2-CLEARANCE, 0, 0), rpy=(0,-np.pi/2,0),
                           fixed=True, collision=False, color=(1,0,0,0.5), parent=BRUSH_NAME)
tpush_b = pscene.create_binder(TOOL_PUSH_NAME, TOOL_PUSH_NAME, FramedTool, point=(0,0,0), rpy=(0,0,0), key=TOOL_PUSH_KEY)


## Table

In [7]:
TABLE_DIM = (0.6, 1.2, 0.7)
TABLE_NAME = "table"
TABLE_LOC = (3, 0, TABLE_DIM[2]/2)
TABLE_RPY = (0,0,0)

In [8]:
table_g = gscene.create_safe(GEOTYPE.BOX, TABLE_NAME, link_name="base_link", dims=TABLE_DIM,
                             center=TABLE_LOC, rpy=TABLE_RPY,
                             fixed=True, collision=True, color=(0.8,0.8,0.8,1))
table_b = pscene.create_binder(TABLE_NAME, TABLE_NAME, PlaceFrame, point=(0,0,TABLE_DIM[2]/2), key=TOOL_PUSH_KEY)
gscene.add_virtual_guardrail(table_g, HEIGHT=0.01, axis="xy")

## Waypoints

In [9]:
WP_NAME = "wp_{}"
WP_DIM = (0.1,0.1,0.01)
WP_LOC = (-0.2, 0.0, TABLE_DIM[2]/2+WP_DIM[2]/2)
WP_RPY = (0,0,0)

In [10]:
wp_g_list = []
for i_t in range(TARGET_COUNT):
    wp_g = gscene.create_safe(GEOTYPE.BOX, WP_NAME.format(i_t), link_name="base_link", dims=WP_DIM,
                              center=(WP_LOC[0], WP_LOC[1]-0.4+i_t*0.2, WP_LOC[2]), rpy=WP_RPY,
                              fixed=True, collision=True, color=(0.0,0.0,0.8,1), parent=TABLE_NAME)
    wp_g_list.append(wp_g)
waypoint_s = pscene.create_subject(oname="waypoints", gname=TABLE_NAME, _type=WaypointTask,
                                 action_points_dict={
                                     wp_g.name: WayFrame(
                                         wp_g.name, wp_g, [0, 0, wp_g.dims[2] / 2], [0, 0, 0])
                                     for wp_g in wp_g_list
                                 })

## Clock

In [11]:
CLOCK_NAME = "clock_{}"
CLOCK_DIM = (0.14,0.05,0.076)
clock_center = np.add(wp_g.center[:2]+(0,), (0,0.1,TABLE_DIM[2]+CLOCK_DIM[2]/2+CLEARANCE))

In [12]:
for i_c in range(CLOCK_COUNT):
    CLOCK_NAME_I = CLOCK_NAME.format(i_c)
    clock_g = gscene.create_safe(GEOTYPE.BOX, CLOCK_NAME_I, link_name="base_link",dims=CLOCK_DIM,
                               center=(clock_center[0], clock_center[1]-i_c*0.4, clock_center[2]), rpy=(0,0,0),
                               fixed=False, collision=True, color=(0.8,0.5,0.3,1))

    push_point_list = []
    for i in range(4):
        R = Rot_axis_series([3, 2], [np.pi/2*i, np.pi/2])
        point = np.round(np.multiply(CLOCK_DIM, -R[:,2])/2, 4)
        point[2] = -CLOCK_DIM[2]/2 + TOOL_PUSH_HEIGHT
        ap_name = "side{}".format(i)
        push_point_list.append(FramePoint(ap_name, clock_g, point=point, rpy=Rot2rpy(R), key=TOOL_PUSH_KEY))

    slide_point = SlidePoint("bottom_p", clock_g, point=(0,0,-CLOCK_DIM[2]/2-CLEARANCE), rpy=(0,0,0), 
                             binded_on=table_b, dist_push_min=0.1, dist_push_max=0.3, dir_push=[-1,0,0], key=TOOL_PUSH_KEY)

    clock_s = pscene.create_subject(oname=CLOCK_NAME_I, gname=CLOCK_NAME_I, _type=PushObject, 
                                  push_point_list=push_point_list, slide_point=slide_point)

## Bag

In [13]:
BAG_NAME = "bag_{}"
BAG_DIM = (0.4, 0.29,0.64)

In [14]:
for i_b in range(BAG_COUNT):
    BAG_NAME_I = BAG_NAME.format(i_b)
    bag_g = gscene.create_safe(GEOTYPE.BOX, BAG_NAME_I, link_name="base_link",dims=BAG_DIM,
                               center=(1.5,0.8*(i_b-int(BAG_COUNT/2)),BAG_DIM[2]/2), rpy=(0,0,0),
                               fixed=False, collision=True, color=(0.8,0.8,0.8,1))
    handle = gscene.create_safe(GEOTYPE.BOX, BAG_NAME_I+"_hdl", link_name="base_link",dims=(0.2,0.03,0.02),
                               center=(0,0,BAG_DIM[2]/2+0.01), rpy=(0,0,0),
                               fixed=False, collision=True, color=(0.6,0.6,0.6,1), parent=BAG_NAME_I)

    push_point_list = []
    for i in range(4):
        R = Rot_axis_series([3, 2], [np.pi/2*i, np.pi/2])
        point = np.round(np.multiply(BAG_DIM, -R[:,2])/2, 4)
        point[2] = -BAG_DIM[2]/2 + MOBILE_PUSH_HEIGHT
        ap_name = "side{}".format(i)
        push_point_list.append(FramePoint(ap_name, bag_g, point=point, rpy=Rot2rpy(R), key=MOBILE_PUSH_KEY))

    slide_point = SlidePoint("bottom_p", bag_g, point=(0,0,-BAG_DIM[2]/2-CLEARANCE), rpy=(0,0,0), 
                                               binded_on=floor_b, dist_push_min=0.5, dist_push_max=1.0, key=MOBILE_PUSH_KEY)

    bag_s = pscene.create_subject(oname=BAG_NAME_I, gname=BAG_NAME_I, _type=PushObject, 
                                  push_point_list=push_point_list, slide_point=slide_point)

### prepare planner

In [15]:
from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene, enable_dual=False)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene, node_trial_max=1000, node_count_max=100, allow_joint_motion=True)
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
gcheck = GraspChecker(pscene)
rcheck = ReachChecker(pscene)
mplan.motion_filters = [gcheck, rcheck]
mplan.incremental_constraint_motion = True

ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
[WARN] reach data is not ready for kmb. Ignoring this robot...


### Custom rules

In [16]:
# joint motion is not allowed when pushing object with holder
idc_push = np.where([stype == PushObject for stype in pscene.subject_type_list])[0]
tplan.explicit_rule = lambda pscene, node, leaf: not any([node[idx]==leaf[idx] and "push" in node[idx] for idx in idc_push])

In [27]:
# generate base pose to reach targets
N_samples = 50
thetas_available = np.array([np.pi/2*i for i in range(4)])

xy_list = floor_b.geometry.get_vertice_radius_from(crob.home_dict)[0][:,:2]
xy_max = np.max(xy_list,axis=0)
xy_min = np.min(xy_list,axis=0)

targets = [pscene.subject_dict[sname] for sname, stype in zip(pscene.subject_name_list, pscene.subject_type_list) if stype == WaypointTask]
wps_dict = {target.oname: [target.action_points_dict[wname] for wname in target.action_points_order] for target in targets}
def gen_reaching_base(state):
    xyt_samples = np.array([np.random.uniform(xy_min, xy_max).tolist()
                            +[np.random.choice(thetas_available)] 
                            for _ in range(N_samples)])

    # check availability for wps
    Twe = np.linalg.inv(brush_b.Toff_lh)
    Tbe_mat = []
    for target in targets:
        i_s = pscene.subject_name_list.index(target.oname)
        ntem = state.node[i_s]
        wps = wps_dict[target.oname][ntem:CUR_TAR] # exclude already reached waypoints
        Tbe_arr = []
        for wp in wps:
            Tbe = np.matmul(wp.get_tf_handle(crob.home_pose), Twe)
            Tbe_arr.append(Tbe)
        Tbe_mat.append(Tbe_arr)
        
    # score with more weights on reachable num on same targets
    score_mat = []
    for i_x, xyt in enumerate(xyt_samples):
        Tbr = gscene.get_tf(ROBOT_BASE, [xyt[0], xyt[1], xyt[2]]+[0]*9)
        score_list = []
        for Tbe_arr in Tbe_mat: # iterate targets
            score_t = 0
            for Tbe in Tbe_arr: # iterate waypoints
                Tre = np.matmul(np.linalg.inv(Tbr), Tbe)
                res = rcheck.check_T(ROBOT_NAME, Tre)
                if res:
                    Q = np.copy(state.Q)
                    Q[:3] = xyt
                    res = mplan.validate_trajectory([Q])
                    score_t += res
            score_list.append(score_t)
        score_mat.append(score_list)
    score_mat = np.square(score_mat) # (pose_num, target_num), square to weight more on large counts
    score_list = np.sum(score_mat, axis=-1,dtype=float)
    score_sum = np.sum(score_list)
    if score_sum>0:
        score_list = np.divide(score_list, score_sum)
        i_select = np.random.choice(range(N_samples), p=score_list)
        Q = np.copy(state.Q)
        Q[:3] = xyt_samples[i_select]
        return Q
    return None

tplan.config_gen = gen_reaching_base

### Plan

In [18]:
gscene.show_pose(crob.home_pose)
chain_list = \
    [BindingChain(BAG_NAME.format(i_b), "bottom_p", "floor_ws", "floor_ws") for i_b in range(BAG_COUNT)] \
    + [BindingChain(CLOCK_NAME.format(i_c), "bottom_p", "table", "table") for i_b in range(CLOCK_COUNT)] \
    + [BindingChain("waypoints", None, None, None)]
initial_state = pscene.initialize_state(crob.home_pose, chain_list=chain_list)
gscene.update_markers_all()
print(pscene.subject_name_list)
print(initial_state.node)

['bag_0', 'bag_1', 'bag_2', 'clock_0', 'clock_1', 'clock_2', 'waypoints']
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 0)


In [28]:
from_state = initial_state.copy(pscene)
snode_schedule_all = []
for i_t in range(1, TARGET_COUNT+1):
    CUR_TAR = i_t
    goal_nodes = [('floor_ws',)*BAG_COUNT+ ('table',)*CLOCK_COUNT + (i_t,)]
    ppline.search(from_state, goal_nodes, max_solution_count=1,
                  verbose=False, display=False, dt_vis=0.01, 
                  timeout=0.5, timeout_loop=100, 
                  multiprocess=True, add_homing=False)
    snode_schedule = tplan.get_best_schedule(at_home=False)
    snode_schedule_all += snode_schedule
    from_state = snode_schedule[-1].state.copy(pscene)
    TextColors.BLUE.println("==================== TARGET {} REACHED ====================".format(i_t))

Use 18/36 agents
Goal reached
Goal reached
======================= terminated 6: required answers acquired  (10.1/100.0) ===============================
======================= terminated 13: Stop called from other agent  (10.1/100.0) ===============================
======================= terminated 12: Stop called from other agent  (10.1/100.0) ===============================
======================= terminated 1: Stop called from other agent  (10.2/100.0) ===============================
======================= terminated 5: Stop called from other agent  (10.2/100.0) ===============================
======================= terminated 2: Stop called from other agent  (10.2/100.0) ===============================
======================= terminated 14: Stop called from other agent  (10.2/100.0) ===============================
======================= terminated 9: Stop called from other agent  (10.2/100.0) ===============================
======================= terminated 16: Stop called fr

======================= terminated 10: Stop called from other agent  (0.3/100.0) ===============================
======================= terminated 11: Stop called from other agent  (0.3/100.0) ===============================
======================= terminated 12: Stop called from other agent  (0.3/100.0) ===============================
======================= terminated 13: Stop called from other agent  (0.3/100.0) ===============================
======================= terminated 14: Stop called from other agent  (0.3/100.0) ===============================
======================= terminated 15: Stop called from other agent  (0.4/100.0) ===============================
======================= terminated 16: Stop called from other agent  (0.4/100.0) ===============================
======================= terminated 17: Stop called from other agent  (0.4/100.0) ===============================
======================= terminated 2: Stop called from other agent  (0.4/100.0) ================

======================= terminated 11: Stop called from other agent  (5.3/100.0) ===============================
======================= terminated 3: Stop called from other agent  (5.4/100.0) ===============================
======================= terminated 16: Stop called from other agent  (5.5/100.0) ===============================
[ERROR] Non-joined subprocesses: [1]
========================== FINISHED (5.7 / 100.0 s) ==============================]
==================== TARGET 3 REACHED ====================
Use 18/36 agents
Goal reached
Goal reached
======================= terminated 11: required answers acquired  (4.2/100.0) ===============================
======================= terminated 0: Stop called from other agent  (4.2/100.0) ===============================
======================= terminated 14: Stop called from other agent  (4.2/100.0) ===============================
======================= terminated 13: Stop called from other agent  (4.2/100.0) =======================

======================= terminated 14: Stop called from other agent  (25.6/100.0) ===============================
======================= terminated 7: Stop called from other agent  (25.6/100.0) ===============================
======================= terminated 11: Stop called from other agent  (25.6/100.0) ===============================
======================= terminated 0: Stop called from other agent  (25.7/100.0) ===============================
======================= terminated 5: Stop called from other agent  (25.7/100.0) ===============================
======================= terminated 6: Stop called from other agent  (25.7/100.0) ===============================
======================= terminated 9: Stop called from other agent  (25.7/100.0) ===============================
======================= terminated 12: Stop called from other agent  (25.8/100.0) ===============================
======================= terminated 16: Stop called from other agent  (25.8/100.0) ===========

In [29]:
pscene.set_object_state(initial_state)
gscene.show_pose(initial_state.Q)

In [30]:
ppline.play_schedule(snode_schedule_all)

('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 0)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 0)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 0)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 0)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 0)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'tpush', 0)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'tpush', 0)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 0)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 0)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 1)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 1)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 1)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 1)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 

### test current again

In [38]:

goal_nodes = [('floor_ws',)*BAG_COUNT+ ('table',)*CLOCK_COUNT + (i_t,)]
ppline.search(from_state, goal_nodes, max_solution_count=1,
              verbose=False, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=100, 
              multiprocess=True, add_homing=False)
snode_schedule = tplan.get_best_schedule(at_home=False)
snode_schedule_all += snode_schedule
from_state = snode_schedule[-1].state.copy(pscene)
TextColors.BLUE.println("==================== TARGET {} REACHED ====================".format(i_t))

Use 18/36 agents
======================= terminated 6: node queue empty 1th  (27.0/100.0) ===============================
======================= terminated 13: node queue empty 2th  (27.0/100.0) ===============================
======================= terminated 15: node queue empty 3th  (27.0/100.0) ===============================
======================= terminated 5: node queue empty 4th  (27.0/100.0) ===============================
======================= terminated 14: node queue empty 5th  (27.0/100.0) ===============================
======================= terminated 17: node queue empty 6th  (27.0/100.0) ===============================
======================= terminated 8: node queue empty 8th  (27.0/100.0) ===============================
======================= terminated 16: node queue empty 8th  (27.0/100.0) ===============================
======================= terminated 12: node queue empty 9th  (27.0/100.0) ===============================
======================= terminat

IndexError: list index out of range

generate table - Geometry
generate table - Handle
generate table - Binder
generate table - Object
generate table - PlanConditions
generate table - PlanList
button clicked
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)->('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)
('floor_ws', 'floor_ws', 'floor_ws', 'table', 'table', 'table', 2)->('floor_ws', 'floor_ws', 'floor_ws', 'tpush', 'table', 'table', 2)
('floor_ws', 'floor